In [2]:
import pandas as pd
import arcpy
import numpy as np

In [24]:
# 2023 needs
needs_2023_source = r'C:\Users\daniel.fourquet\Documents\Tasks\VTrans Update\2023-needs\2023_VTrans_MidTerm_Needs.gdb\VTrans_MidTerm_Needs_2023'

# 2021 needs
needs_2021_source = r'C:\Users\daniel.fourquet\Documents\Tasks\VTrans Update\2023-needs\A1 - Common Datasets\Previous Needs Layer\2021_VTrans_Mid_term_Needs.gdb\tbl_2021_needs_Dissolve_events'

In [4]:
needs_fields_2021 = [
 'CoSS_congestion',
 'CoSS_reliability',
 'CoSS_Rail_Reliability',
 'CoSS_capacity_preservation',
 'CoSS_LA_TDM',
 'CoSS_non_LA_TDM',
 'RN_congestion',
 'RN_reliability',
 'RN_Capacity_Preservation',
 'RN_LA_TDM',
 'RN_non_LA_TDM',
 'RN_transit_equity',
 'RN_AC_Transit_Access',
 'RN_AC_Bicycle_Access',
 'RN_AC_pedestrian_access',
 'UDA_road_capacity',
 'UDA_road_ops',
 'UDA_transit_freq',
 'UDA_transit_ops',
 'UDA_transit_capacity',
 'UDA_transit_facilities',
 'UDA_street_grid',
 'UDA_bike_infrast',
 'UDA_ped_infrast',
 'UDA_comp_street',
 'UDA_safety_feat',
 'UDA_onstreet_park',
 'UDA_offstreet_park',
 'UDA_intersection_des',
 'UDA_signage',
 'UDA_traffic_calm',
 'UDA_landscape',
 'UDA_sidewalk',
 'RN_Growth_Area',
 'IEDA',
 'Safety_Segments',
 'CoSS_Safety_Segments',
 'Safety_Intersection',
 'CoSS_Safety_Intersection',
 'Safety_Pedestrian'
]

In [5]:
needs_fields_2023 = [
 'CoSS_Congestion',
 'CoSS_Reliability',
 'CoSS_Rail_Reliability',
 'CoSS_Capacity_Preservation',
 'CoSS_LA_TDM',
 'CoSS_non_LA_TDM',
 'CoSS_Safety_Intersection',
 'CoSS_Safety_Segments',
 'RN_Congestion',
 'RN_Reliability',
 'RN_Capacity_Preservation',
 'RN_LA_TDM',
 'RN_non_LA_TDM',
 'RN_AC_Bicycle_Access',
 'RN_AC_Pedestrian_Access',
 'RN_AC_Transit_Access',
 'RN_Transit_Equity',
 'Safety_Segments',
 'Safety_Intersection',
 'Safety_Pedestrian',
 'IEDA',
 'UDA_Bike_Infrast',
 'UDA_Comp_Street',
 'UDA_Intersection_Des',
 'UDA_Landscape',
 'UDA_Offstreet_Park',
 'UDA_Onstreet_Park',
 'UDA_Ped_Infrast',
 'UDA_Road_Capacity',
 'UDA_Road_Ops',
 'UDA_Safety_Feat',
 'UDA_Sidewalk',
 'UDA_Signage',
 'UDA_Street_Grid',
 'UDA_Traffic_Calm',
 'UDA_Transit_Capacity',
 'UDA_Transit_Facilities',
 'UDA_Transit_Freq',
 'UDA_Transit_Ops',
 'RN_Growth_Area'
]

In [6]:
# Dictionary to convert 2021 needs to 2023 needs for consistency
dict_convert_schema = {
    'CoSS_congestion': 'CoSS_Congestion',
    'CoSS_reliability': 'CoSS_Reliability',
    'CoSS_Rail_Reliability': 'CoSS_Rail_Reliability',
    'CoSS_capacity_preservation': 'CoSS_Capacity_Preservation',
    'CoSS_LA_TDM': 'CoSS_LA_TDM',
    'CoSS_non_LA_TDM': 'CoSS_non_LA_TDM',
    'CoSS_Safety_Intersection': 'CoSS_Safety_Intersection',
    'CoSS_Safety_Segments': 'CoSS_Safety_Segments',
    'RN_congestion': 'RN_Congestion',
    'RN_reliability': 'RN_Reliability',
    'RN_Capacity_Preservation': 'RN_Capacity_Preservation',
    'RN_LA_TDM': 'RN_LA_TDM',
    'RN_non_LA_TDM': 'RN_non_LA_TDM',
    'RN_AC_Bicycle_Access': 'RN_AC_Bicycle_Access',
    'RN_AC_pedestrian_access': 'RN_AC_Pedestrian_Access',
    'RN_AC_Transit_Access': 'RN_AC_Transit_Access',
    'RN_transit_equity': 'RN_Transit_Equity',
    'Safety_Segments': 'Safety_Segments',
    'Safety_Intersection': 'Safety_Intersection',
    'Safety_Pedestrian': 'Safety_Pedestrian',
    'IEDA': 'IEDA',
    'UDA_bike_infrast': 'UDA_Bike_Infrast',
    'UDA_comp_street': 'UDA_Comp_Street',
    'UDA_intersection_des': 'UDA_Intersection_Des',
    'UDA_landscape': 'UDA_Landscape',
    'UDA_offstreet_park': 'UDA_Offstreet_Park',
    'UDA_onstreet_park': 'UDA_Onstreet_Park',
    'UDA_ped_infrast': 'UDA_Ped_Infrast',
    'UDA_road_capacity': 'UDA_Road_Capacity',
    'UDA_road_ops': 'UDA_Road_Ops',
    'UDA_safety_feat': 'UDA_Safety_Feat',
    'UDA_sidewalk': 'UDA_Sidewalk',
    'UDA_signage': 'UDA_Signage',
    'UDA_street_grid': 'UDA_Street_Grid',
    'UDA_traffic_calm': 'UDA_Traffic_Calm',
    'UDA_transit_capacity': 'UDA_Transit_Capacity',
    'UDA_transit_facilities': 'UDA_Transit_Facilities',
    'UDA_transit_freq': 'UDA_Transit_Freq',
    'UDA_transit_ops': 'UDA_Transit_Ops',
    'RN_Growth_Area': 'RN_Growth_Area'
}

In [25]:
def get_mileage(df, field, district=None):
    if district:
        filter = df.loc[(df[field] == 'YES') & (df.loc[df['VDOT_District'] == district])]
    else:
        filter = df.loc[df[field] == 'YES']
        
    mileage = filter['Segment_Length'].sum()
    return round(mileage, 1)

In [26]:
# Make dataframe for 2021 needs
fields_2021 = [field.name for field in arcpy.ListFields(needs_2021_source)]
df_2021 = pd.DataFrame([row for row in arcpy.da.SearchCursor(needs_2021_source, fields_2021)], columns=fields_2021)
df_2021['Segment_Length'] = abs(df_2021['To_Measure'] - df_2021['From_Measure'])
df_2021.rename(columns=dict_convert_schema, inplace=True)

# Make dataframe for 2023 needs
fields_2023 = [field.name for field in arcpy.ListFields(needs_2023_source)]
df_2023 = pd.DataFrame([row for row in arcpy.da.SearchCursor(needs_2023_source, fields_2023)], columns=fields_2023)

In [27]:
# Calculate 2021 mileage
mileage_by_need_2021 = [(need, get_mileage(df_2021, need)) for need in needs_fields_2023]
df_mileage_2021 = pd.DataFrame(mileage_by_need_2021, columns=['Need', 'Mileage'])

# Calculate 2023 schema mileage
mileage_by_need_2023 = [(need, get_mileage(df_2023, need)) for need in needs_fields_2023]
df_mileage_2023 = pd.DataFrame(mileage_by_need_2023, columns=['Need', 'Mileage'])

In [58]:
# Merge mileage and calculate differences
df_mileage_comparison = df_mileage_2021.merge(df_mileage_2023, on='Need', suffixes=('_2021', '_2023'))
df_mileage_comparison.rename(columns={'Mileage_2021': '2021 Mileage', 'Mileage_2023': '2023 Mileage'}, inplace=True)
df_mileage_comparison['Difference'] = df_mileage_comparison['2023 Mileage'] - df_mileage_comparison['2021 Mileage']
df_mileage_comparison['% Diff'] = round(df_mileage_comparison['Difference'] / ((df_mileage_comparison['2023 Mileage'] + df_mileage_comparison['2021 Mileage']) / 2), 2)
df_mileage_comparison

,Need,2021 Mileage,2023 Mileage,Difference,% Diff
0,CoSS_Congestion,810.8,745.5,-65.3,-0.08
1,CoSS_Reliability,240.4,156.9,-83.5,-0.42
2,CoSS_Rail_Reliability,2004.5,2176.8,172.3,0.08
3,CoSS_Capacity_Preservation,3621.1,3638.9,17.8,0.00
4,CoSS_LA_TDM,2898.5,2719.1,-179.4,-0.06
5,CoSS_non_LA_TDM,4001.1,6438.1,2437.0,0.47
6,CoSS_Safety_Intersection,143.1,162.3,19.2,0.13
7,CoSS_Safety_Segments,2539.0,2266.7,-272.3,-0.11
8,RN_Congestion,1088.5,923.9,-164.6,-0.16
9,RN_Reliability,332.0,226.5,-105.5,-0.38


In [59]:
# Calcualte mileage comparison by district

# Get district list
df_2021.rename(columns={'VDOT_Districts': 'VDOT_District'}, inplace=True)
districts = df_2021['VDOT_District'].values.tolist()
districts = list(set(districts))
districts.remove(None)
districts.sort()
districts

# Calculate mileage
for district in districts:
    print(f'Calculating mileage for {district} District')
    df_district_2021 = df_2021.loc[df_2021['VDOT_District'] == district]
    district_mileage_2021 = [(need, get_mileage(df_district_2021, need)) for need in needs_fields_2023]
    df_district_mileage_2021 = pd.DataFrame(district_mileage_2021, columns=['Need', 'Mileage'])

    df_district_2023 = df_2023.loc[df_2023['VDOT_District'] == district]
    district_mileage_2023 = [(need, get_mileage(df_district_2023, need)) for need in needs_fields_2023]
    df_district_mileage_2023 = pd.DataFrame(district_mileage_2023, columns=['Need', 'Mileage'])

    df_district_mileage_comparison = df_district_mileage_2021.merge(df_district_mileage_2023, on='Need', suffixes=('_2021', '_2023'))
    df_district_mileage_comparison.rename(columns={'Mileage_2021': '2021 Mileage', 'Mileage_2023': '2023 Mileage'}, inplace=True)
    df_district_mileage_comparison['Difference'] = df_district_mileage_comparison['2023 Mileage'] - df_district_mileage_comparison['2021 Mileage']
    df_district_mileage_comparison['% Diff'] = round(df_district_mileage_comparison['Difference'] / ((df_district_mileage_comparison['2023 Mileage'] + df_district_mileage_comparison['2021 Mileage']) / 2), 2)
    df_district_mileage_comparison.rename(columns={'2021 Mileage': f'{district} 2021 Mileage', '2023 Mileage': f'{district} 2023 Mileage', '% Diff': f'{district} % Diff', 'Difference': f'{district} Difference'}, inplace=True)

    df_mileage_comparison = df_mileage_comparison.merge(df_district_mileage_comparison, on='Need')


Calculating mileage for Bristol District
Calculating mileage for Culpeper District
Calculating mileage for Fredericksburg District
Calculating mileage for Hampton Roads District
Calculating mileage for Lynchburg District
Calculating mileage for Northern Virginia District
Calculating mileage for Richmond District
Calculating mileage for Salem District
Calculating mileage for Staunton District


In [60]:
df_mileage_comparison.to_clipboard(index=False)